In [3]:

import datetime

from dictionarydatabase import DictionaryDataBase

In [4]:
db = DictionaryDataBase(fields={
    ("name", "nm", str): (lambda x: x["name"]),
    ("caught", "cg", datetime.datetime): (lambda x: x["caught"]),
    ("height", "hg", float): (lambda x: x["dimensions"]["height"]),
    ("legs", "lg", int): (lambda x: x["appendages"]["legs"]),
    ("arms", "ar", int): (lambda x: x["appendages"]["arms"]),
    ("flys", "fy", bool): (lambda x: x["abilities"]["flys"]),
}) 

data = [
    {
        "name": "zebra", "caught": datetime.datetime(2010, 3, 15),
        "dimensions": {"height": 1.4, "length": 1.8}, 
        "appendages": {"legs": 4}
    },
    {
        "name": "monkey", "caught": datetime.datetime(2002, 4, 11),
        "dimensions": {"height": 1.2}, 
        "appendages": {"legs": 2, "arms": 2}
    },
    {
        "name": "duck", "caught": datetime.datetime(1987, 11, 3),
        "dimensions": {"height": 0.15, "length": 0.24}, 
        "appendages": {"legs": 2, "wings": 2}, 
        "abilities": {"flys": True}
    },
    {
        "name": "whale", "caught": datetime.datetime(1910, 1, 15),
        "dimensions": {"height": 2.1, "length": 5.6}, 
        "appendages": {"legs": 2}
    },
]

db.set_data(data)

db.query(">2000-01-01[caught] or t[flys]")

[{'name': 'duck',
  'caught': datetime.datetime(1987, 11, 3, 0, 0),
  'dimensions': {'height': 0.15, 'length': 0.24},
  'appendages': {'legs': 2, 'wings': 2},
  'abilities': {'flys': True}},
 {'name': 'zebra',
  'caught': datetime.datetime(2010, 3, 15, 0, 0),
  'dimensions': {'height': 1.4, 'length': 1.8},
  'appendages': {'legs': 4}},
 {'name': 'monkey',
  'caught': datetime.datetime(2002, 4, 11, 0, 0),
  'dimensions': {'height': 1.2},
  'appendages': {'legs': 2, 'arms': 2}}]

In [6]:
class SQLDataBase(DataBase):
    
    def __init__(self, field_names):
        super().__init__()
        self.field_names = {k.lower(): v.lower() for k, v in field_names.items()}
        
    def _get_field_full_name(self, field_name):
        m = field_name.lower()
        for abbr, full in self.field_names.items():
            if m == abbr or m == full:
                return full
        return m
    
    # -------------------------------------------------------------------------
    # Database specific search actions.
    # -------------------------------------------------------------------------
    
    def search_not(self, operands):
        return 'NOT ' + operands.evaluate()

    def search_and(self, operands):
        return "(" + " AND ".join([oper.evaluate() for oper in operands]) + ")"
    
    def search_or(self, operands):
        return "(" + " OR ".join([oper.evaluate() for oper in operands]) + ")"
    
    def search_string(self, term, field):
        nom = self._get_field_full_name(field)
        x = term.replace("*", "%")
        return f"\"{nom}\" LIKE '{x}'"
        
    def search_number(self, comp, term, field):
        nom = self._get_field_full_name(field)
        return f"\"{nom}\" {comp} {term}"

    def search_list(self, comp, term, field):
        return ""
    

db = SQLDataBase(field_names={"bk": "bucket", "an": "anchor"})

db.query("empty[bk] and (>6[an] or light[an])")

'("bucket" LIKE \'empty\' AND ("anchor" > 6 OR "anchor" LIKE \'light\'))'